In [55]:
import pandas as pd
import sqlite3

In [56]:
areas = pd.read_sql_query('SELECT * FROM Areas_area', sqlite3.connect('../Server/db.sqlite3'))
areas

,id,value,label
0,1,1,LAMCE


In [57]:
devices = pd.read_sql_query('SELECT * FROM Devices_device', sqlite3.connect('../Server/db.sqlite3'))
devices

,variables,id,esp_id,timestamp,area,master,node
0,"[""temperature"", ""humidity""]",1,08AB29DB511,2025-07-21 10:44:01.824829,1,0,Pier
1,"[""pressure"", ""humidity""]",2,B80DB629E74,2025-09-03 18:37:41.927310,1,1,GRVA


In [62]:
def getInfo(id):
    target = devices.loc[devices.esp_id == id]

    if len(target) == 0:
        return None

    node   = devices.iloc[0].node
    area   = devices.iloc[0].area
    target = areas.loc[areas.value == area]
    
    if len(areas) == 0:
        return None
    
    label = target.iloc[0].label
    return {'area': int(area), 'label': label, 'node': node}


getInfo('08AB29DB511')

{'area': 1, 'label': 'LAMCE', 'node': 'Pier'}